In [ ]:
import telepot
from telepot.loop import MessageLoop
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton, ReplyKeyboardMarkup, KeyboardButton
from telepot.delegate import (per_chat_id, per_application, call, create_open, pave_event_space)
from pprint import pprint
import sys
import time
import pandas as pd
import numpy as np
%run get_results.ipynb

<h3>Simulate a database to store unread messages</h3>

In [ ]:
# Simulate a database to store unread messages
class MessageStore(object):
    def __init__(self):
        self._db = {}
        self._results_db = []
        self._results_ontology_db = []
        self.tag = []
        self.flagFine = False
        
    def put_flag(self, f):
        self.flagFine = f
        
    def pull_flag(self):
        return self.flagFine
     
    def put_tag(self, t):
        if len(self.tag):
            #del self.tag
            self.tag.append(t)
        else:
            self.tag.append(t)
       
    def pull_tag(self):
        t = self.tag
        return t 
    
    def put_results(self, results):
        if len(self._results_db):
            del self._results_db[0]
        self._results_db.append(results)
            
    def pull_results(self):
        if len(self._results_db):
            res = self._results_db
            return res
        else:
            return []

    def put_results_ontology(self, results):
        if len(self._results_ontology_db):
            del self._results_ontology_db[0]
        self._results_ontology_db.append(results)
            
    def pull_results_ontology(self):
        if len(self._results_ontology_db):
            res1 = self._results_ontology_db
            return res1
        else:
            return []    
        
    def put(self, msg):
        chat_id = msg['chat']['id']

        if chat_id not in self._db:
            self._db[chat_id] = []

        self._db[chat_id].append(msg)

    # Pull all unread messages of a `chat_id`
    def pull(self, chat_id):
        
        if len(self._db):
            messages = self._db[chat_id]
            # sort by date
            messages.sort(key=lambda m: m['date'])
            return messages
        else:
            return []
    
    def delete_mess(self, chat_id):
        if len(self._db):
            del self._db[chat_id]

<h2>Control Telegram Bot</h2>

In [ ]:
class MessageHandler(telepot.helper.ChatHandler):
    def __init__(self, seed_tuple, store, **kwargs):
        super(MessageHandler, self).__init__(seed_tuple, **kwargs)
        self._store = store

    def _read_messages(self, messages):
        for msg in messages:
            # assume all messages are text
            self.sender.sendMessage(msg['text'])

    def on_chat_message(self, msg):
        
        #Flag per non entrare nella iterazione del yes or no quando si sta processando per la prima volta il problema 
        flag = True
        
        flagCheckTec = False
        
        content_type, chat_type, chat_id = telepot.glance(msg)
        
        if content_type == 'text' :
            name = msg["from"]["first_name"]
            txt = str(msg['text'])
        
        else:
            self.sender.sendMessage("I don't understand")
            return
        
        #Con questo metodo creiamo una lista con i messaggi precedenti relativi alla chat con l'utente corrente 
        message_history = self._store.pull(chat_id)
          
        #Se il comando è start mando il messaggio di inzio ed elimino i messaggi precedenti salvati relativi all'utente 
        if '/start' == txt:      
            markup = ReplyKeyboardMarkup(keyboard=[['Word2vec', 'Tfidf']])
            self.sender.sendMessage('Hello {}! I\'m StackBot.\n I\'m here to try to help you find the solutions to yours problems around the python world among the Stack Overflow questions.\nI can do that using two different technologies, choose one from below.'.format(name), reply_markup = markup)
            
            self._store.put_flag(False)
            self._store.delete_mess(chat_id)
            self._store.put(msg)
            
            #Ogni volta che viene letto il comando /start viene resettata la chat è pulito il'db' dei messaggi
            if len(message_history) > 0 and txt == '/start':
                self._store.delete_mess(chat_id)
                message_history = self._store.pull(chat_id) 
                self._store.put(msg)
            
        #Controllo comandi errati
        elif len(message_history) == 0 and txt != '/start':
            
            markup = ReplyKeyboardMarkup(keyboard=[['/start']])
            
            self.sender.sendMessage('You have to write /start to begin the interaction', 
                                    reply_markup = markup)

        
        if (len(message_history) == 1 and message_history[0]['text'] == '/start'):
            
            #controlliamo che il messaggio appena inviato non sia un altro comando 
            if 'entities' in msg:
                
                self._store.delete_mess(chat_id)
                self.sender.sendMessage("You can't type this in your request, please restart your interaction")
        
            else:
                
                if ((txt != 'Word2vec') and (txt != 'Tfidf')):
                    
                    markup = ReplyKeyboardMarkup(keyboard=[['Word2vec', 'Tfidf']])
                    self.sender.sendMessage('Your input is wrong')
                    self.sender.sendMessage('I can do that using two different technologies, choose one from below.', reply_markup = markup)

                if ((txt == 'Word2vec') or (txt == 'Tfidf')):
                    
                    self._store.put(msg)
                    self.sender.sendMessage('Tell me something about your problem involving python language, try to be as general as possible, example: Problem loading csv file\nI will propose to you some keyword from stack overflow question and you need to tell me if that word has something to do with your problem.\nNow i will do everything using the {} technology.'.format(message_history[1]['text']), reply_markup = None)
        
        #Se l'unico messaggio salvato è il comando start procediamo con il primo passo del processo
        if (len(message_history) == 3 and message_history[0]['text'] == '/start') :
            
            #controlliamo che il messaggio appena inviato non sia un altro comando 
            if 'entities' in msg:
                self._store.delete_mess(chat_id)
                self.sender.sendMessage("You can't type this in your request, please restart your interaction")
                
           
            else:
                #Ricerca input con w2v
                if (message_history[1]['text'] == 'Word2vec'):
                    
                    print('Searching the input string :' + txt + '\n')
                    print('Using W2V tecnology')
                    print('-'*100)
                    
                    results = w2v_stack_result(txt)                 
                    self._store.put_results(results)
                    
                #Ricerca input con tf-idf
                if (message_history[1]['text'] == 'Tfidf'):
                    
                    print('Searching the input string :' + txt + '\n')
                    print('Using tf-idf tecnology')
                    print('-'*100)
                    
                    results = tfidf_stack_result(txt)
                    self._store.put_results(results)
                
                #Salviamo nel nostro "db" i risultati del w2v o tf-idf e il messaggio relativo al problema dell'utente
               
                self._store.put(msg)
                
                #Richiamo funzione per rendere output all'utente      
                sendControl(self, results, self._store, message_history, 0, None, self._store.pull_flag())
                
                flag = False
    
        #Gestione risposte utente durante interazione con il bot
        
        if len(message_history) > 1 and flag:
            
            res = []
            
            if txt == 'Yes' or txt == 'yes':  
                #Richiamo funzione che filtra i risultati
                res = filter_results(self._store,1)
                self._store.put_results(res)
                sendControl(self, res, self._store, message_history, 1, None, self._store.pull_flag())
                
            elif txt == 'No' or txt == 'no':
                #Richiamo funzione che filtra i risultati
                res = filter_results(self._store,2)
                self._store.put_results(res)
                sendControl(self, res, self._store, message_history, 2, None, self._store.pull_flag())
                
            elif txt == 'Continue' or txt == 'continue':
                res = self._store.pull_results()
                #Elimino la risposta che l'utente ha scartato
                res[0].pop(0)
                self._store.put_results(res[0])
                sendControl(self, res[0], self._store, message_history, 5, None, self._store.pull_flag())
            
            elif txt == 'I don\'t know' or txt == 'i don\'t know':
                #Richiamo funzione che filtra i risultati
                res = filter_results(self._store,3)
                self._store.put_results(res)
                sendControl(self, res, self._store, message_history, 3, None, self._store.pull_flag())
                
            #L'utente ha trovato la risposta al suo problema   
            elif txt == 'I\'m fine' or txt == 'i\'m fine':
                res = self._store.pull_results()
                
                #Se ha trovato la risposta utilizzando solo la prima tecnologia scelta
                if (self._store.pull_flag() == False):
                    #Se la tecnologia è word2vec
                    if(message_history[1]['text'] == 'Word2vec'):
                        #Cerchiamo un match tra il titolo della risposta e le funzione del db di Code Ontology
                        results_ontology = w2v_ontology_result(str(res[0][3][3]))
                        print("textINPUT_w2v_ONT", str(res[0][3][3] + '\n'))
                    else:
                        results_ontology = tfidf_onto_result(str(res[0][3][3]))
                        print("textINPUT_ TF-IDF_ONT", str(res[0][3][3]) + '\n')
                #Se la risposta è stata trovata con la seconda tecnologia 
                else:
                    
                    if(message_history[1]['text'] == 'Word2vec'):
                        #Cerchiamo un match tra il titolo della risposta e le funzione del db di Code Ontology
                        results_ontology = tfidf_onto_result(str(res[0][3][3]))
                        print("textINPUT_W2V_ONT", str(res[0][3][3]) + '\n')
                    else:
                        #Cerchiamo un match tra il titolo della risposta e le funzione del db di Code Ontology
                        results_ontology = w2v_ontology_result(str(res[0][3][3]))
                        print("textINPUT_ TF-IDF_ONT", str(res[0][3]) + '\n')
                    
                print("Code Ontology Results:", results_ontology)
                print('-'*100)
                
                sendControl(self, res, self._store, message_history, 4, results_ontology, self._store.pull_flag())
            
            #Se non è stata trovata risposta utilizzando word to vec
            elif txt == 'Try with Tfidf' or txt == 'try with tfidf':
                
                self._store.put_flag(True)
                
                print('changing technology - tfidf \n')
                print('searching the string: ' + message_history[3]['text'] + '\n')
                
                #calcolo il tfidf con la stringa del problema inserita dall'utente
                results = tfidf_stack_result(message_history[3]['text'])
                
                #Vado a salvare il nuovo results su cui ciclare
                self._store.put_results(results)
                
                #Richiamo funzione per rendere output all'utente con l'altra tecnologia
                sendControl(self, results, self._store, message_history, 0, None, self._store.pull_flag())
            
            #Se non è stata trovata risposta utilizzando tf-idf
            elif txt == 'Try with Word2vec' or txt == 'try with word2vec':    
                self._store.put_flag(True)
                
                #calcolo il w2v con la stringa del problema inserita dall'utente
                results = w2v_stack_result(message_history[3]['text'])
                
                #Vado a salvare i nuovi results su cui ciclare
                self._store.put_results(results)
                
                #Richiamo funzione per rendere output all'utente con l'altra tecnologia
                sendControl(self, results, self._store, message_history, 0, None, self._store.pull_flag())
            
            self._store.put(msg)
            
                

In [ ]:
#Gestione array risultati 
def checkLenResults(results):
    
    if not results:
        #Ossia che results è vuoto --> []
        return True
    else:
        #Ossia che results non è vuoto --> [[..], [..]]
        return False

In [ ]:
#Gestione cambio tecnologia e reset iterazione
def checkMesTec(self, flag, message):
    markup1 = ReplyKeyboardMarkup(keyboard=[[KeyboardButton(text='/start')]])
    markup3 = ReplyKeyboardMarkup(keyboard=[[KeyboardButton(text='Try with Tfidf')]])
    markup4 = ReplyKeyboardMarkup(keyboard=[[KeyboardButton(text='Try with Word2vec')]])
    
    if (flag):
        self.sender.sendMessage("You have already tried both technologies.\nIf you haven't found a question that suit your problem you can type /start to restart the bot and try to reformulate your problem", reply_markup = markup1)            
    else:
        if ( message == 'Word2vec'):
            self.sender.sendMessage("There aren't other soluctions with this technology, you click 'Try with Tfidf' if you want try with Tfidf", reply_markup = markup3)
        else:
            self.sender.sendMessage("There aren't other soluctions with this technology, you click 'Try with Word2vec' if you want try with Word2vec", reply_markup = markup4)

In [ ]:
#Gestione dell'interazione con l'utente
def sendControl(self, results, selfStore, message_history, typeTxt, results_ontology, flagCheckTec):
    
    markup = ReplyKeyboardMarkup(keyboard=[[KeyboardButton(text='Continue'), 'I\'m fine']])
    markup1 = ReplyKeyboardMarkup(keyboard=[[KeyboardButton(text='/start')]])
    markup2 = ReplyKeyboardMarkup(keyboard=[[ 'Yes' ,KeyboardButton(text='No'), 'I don\'t know']])

    #Gestione input utente iniziale(problema utente) 
    if (typeTxt == 0):
        
        print("RESULTS: ", results)
        print('-'*100)
        
        #Dai risultati prelevo tutte le liste dei tag delle varie domande 
        tags = [el[2] for el in results]
        #Restituire primo tag della prima domanda di Stack
        self.sender.sendMessage("Your question has something to do with: " + tags[0][0] + "?", reply_markup = markup2)
        self.sender.sendMessage("Write 'Yes' for yes or 'No' for no or 'I don\'t know' if you don\'t know")
    
    #Se l'utente ha cliccato YES (quindi è interessato al tag proposto)
    if (typeTxt == 1):
        
        #Restituire la domanda in prima posizione nei risultati 
        self.sender.sendMessage("The solution to your problem is likely to have been discussed on this page: https://stackoverflow.com/questions/" + str(results[0][0]) + "/")
        self.sender.sendMessage("Click 'Continue' if you want continue the search or click 'I\'m fine' if you want to stop", reply_markup = markup)
    
    #Se l'utente ha cliccato NO (quindi non è interessato al tag proposto)
    if (typeTxt == 2):
        
        #Se vi sono ancora delle domande nell'array dei risultati
        if (checkLenResults(results) == False):
            
            #Dai risultati prelevo tutte le liste dei tag delle varie domande 
            tags = [el[2] for el in results]
            
            #Restituire nuovo tag della domanda successiva
            self.sender.sendMessage("Your question has something to do with:" + tags[0][0] + "?", reply_markup = markup2)
            self.sender.sendMessage("Write 'Yes' for yes or 'No' for no or 'I dont\'t know' if you have doubts")
        
        #Se non vi sono più domande nell'array dei risultati
        else:
            #gestione cambio tecnologie
            checkMesTec(self, flagCheckTec , message_history[1]['text'])

                
    #Se l'utente ha cliccato I'DONT KNOW (quindi non sa se è interessato al tag proposto)
    if (typeTxt == 3):
            
        #Se vi sono ancora delle domande
        if (checkLenResults(results) == False):
            
            #Dai risultati prelevo tutte le liste dei tag delle varie domande 
            tags = [el[2] for el in results]
            
            #Restuire nuovo tag della stessa domanda
            self.sender.sendMessage("Your question has something to do with:" + tags[0][0] + "?", reply_markup = markup2)
            self.sender.sendMessage("Write 'Yes' for yes or 'No' for no or 'I dont\'t know' if you have doubts")
        
        #Se non vi sono più domande nell'array dei risultati
        else:
            #gestione cambio tecnologie
            checkMesTec(self, flagCheckTec , message_history[1]['text'])
            
     
    #Se l'utente ha cliccato I'M FINE (ha trovato la sua risposta)
    if (typeTxt == 4):
        print('Ontology Results:', results_ontology + '\n')
        #Rendiamo la funzione dal db di Code Ontology con la descrizione più simile al titolo del post scelto come risposta dall'utente
        #in base alla tecnologia usata  
        self.sender.sendMessage("The possible function solution in Java is:   " + str(results_ontology[0][0]) + "\n\nThe its description is: " + str(results_ontology[0][1]))
        
    #Se l'utente ha cliccato CONTINUE (scarta la domanda proposta)
    if (typeTxt == 5):
        
        #Se vi sono ancora delle domande
        if (checkLenResults(results) == False):
        
            #Dai risultati prelevo tutte le liste dei tag delle varie domande 
            tags = [el[2] for el in results]
            
            #Restituire primo tag della prima domanda di Stack
            self.sender.sendMessage("Your question has something to do with: " + tags[0][0] + "?", reply_markup = markup2)
            self.sender.sendMessage("Write 'Yes' for yes or 'No' for no or 'I don\'t know' if you don\'t know")
        
        #Se non vi sono più domande nell'array dei risultati 
        else:
            #gestione cambio tecnologie
            checkMesTec(self, flagCheckTec , message_history[1]['text'])
                

In [ ]:
#Filtraggio risultati
def filter_results(store, choice):
        #Carico in variabili i precedenti risultati
        tmp_results = store.pull_results()
        results = tmp_results[0]
        
        tags = []
        tmp_results1 = []
        tmp_results2 = []    
        
        #Da tutte le domande prelevo tutte le liste dei tag
        tags = [el[2] for el in results]

        i = 0
          
        if len(tags) > 0:
            #choice == 1 vuol dire che l'utente è interessato al tag
            if choice == 1: #Y
                
                tag_target = tags[0][0]
                
                #Metto in cima le risposte che hanno il tag "tag_target"
                for item in results:
                    if tag_target not in item[2]:
                        tmp_results1.append(item)
                    else:
                        tmp_results2.append(item)
                        
                results.clear()                        
                results = tmp_results2 + tmp_results1
                
            #choice == 2 vuol dire che l'utente non è interessato al tag   
            if choice == 2: #N
                
                #Metto in cime le ripsoste che non hanno il tag "tag_target"
                
                for item in results:
                    if tags[0][0] not in item[2]:
                        tmp_results1.append(item)
                    else:
                        tmp_results2.append(item)
                
                results = tmp_results1 + tmp_results2
                
                #Elimino il tag "tag_target" in tutte le potenziali risposte
                
                j = 0
               
                print("Results after selecting No for a tag: ", results)
                print('-'*100)
                
                t = tags[0][0]
                    
                for item in results:
                    if t in item[2]:
                        results[j][2].remove(t)
                    j = j+1    
                
                #Elimino tutte le domande che non hanno tag, ossia []
                
                tmp = []
                for item in results:
                    if item[2]:
                        tmp.append(item)
                results.clear()
                results = tmp
                
            if choice == 3: #I don\'t know
                #ELIMINO QUEL TAG DA SOLO QUELLA DOMANDA
                results[0][2].remove(tags[0][0])
                
                #Elimino tutte le domande che non hanno tag, ossia []
                
                tmp = []
                for item in results:
                    if item[2]:
                        tmp.append(item)
                results.clear()
                results = tmp

        
        return results